In [1]:
import requests
from pandas import json_normalize
import psycopg2 as pg
import pandas.io.sql as psql
from sklearn import preprocessing

In [2]:
le = preprocessing.LabelEncoder()

In [4]:
url = 'http://bootcamp-ml.westus2.cloudapp.azure.com:5000/data'

response = requests.get(url)
dictr = response.json()
df = json_normalize(dictr)
#print(df)

In [5]:
df.columns

Index(['age', 'radiologists_assessment', 'cancer_diagnosis',
       'comparison_mammogram', 'patients_birads', 'family_history',
       'hormone_therapy', 'indicator', 'breast_biopsy', 'mammogram',
       'cancer_type', 'body_mass', 'patients_id'],
      dtype='object')

In [6]:
dataframeWF = df

In [7]:
le.fit(dataframeWF["patients_birads"].unique())
dataframeWF["patients_birads"] = le.transform(dataframeWF["patients_birads"])


In [8]:
le.fit(dataframeWF["breast_biopsy"].unique())
dataframeWF["breast_biopsy"] = le.transform(dataframeWF["breast_biopsy"])


In [9]:
le.fit(dataframeWF["comparison_mammogram"].unique())
dataframeWF["comparison_mammogram"] = le.transform(dataframeWF["comparison_mammogram"])


In [10]:
le.fit(dataframeWF["family_history"].unique())
dataframeWF["family_history"] = le.transform(dataframeWF["family_history"])


In [11]:
le.fit(dataframeWF["hormone_therapy"].unique())
dataframeWF["hormone_therapy"] = le.transform(dataframeWF["hormone_therapy"])


In [12]:
le.fit(dataframeWF["cancer_diagnosis"].unique())
dataframeWF["cancer_diagnosis"] = le.transform(dataframeWF["cancer_diagnosis"])


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframeWF[["age","patients_birads","breast_biopsy","comparison_mammogram","family_history","hormone_therapy"]], dataframeWF["cancer_diagnosis"], random_state=42)


In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.9931

In [15]:
importances = rf.feature_importances_
importances

array([0.59621808, 0.11997112, 0.06062927, 0.06967839, 0.06386374,
       0.0896394 ])

In [17]:
sorted(zip(rf.feature_importances_, ["age","patients_birads","breast_biopsy","comparison_mammogram","family_history","hormone_therapy"]), reverse=True)


[(0.5962180826636313, 'age'),
 (0.1199711156938941, 'patients_birads'),
 (0.08963940244873803, 'hormone_therapy'),
 (0.06967839388977175, 'comparison_mammogram'),
 (0.06386373801799476, 'family_history'),
 (0.060629267285970026, 'breast_biopsy')]

In [18]:
import pandas as pd
prediction=pd.DataFrame(rf.predict(X_test))

In [19]:
test = pd.DataFrame(X_test)
test.shape

(10000, 6)

In [20]:
prediction.shape

(10000, 1)

In [21]:
prediction[0].unique()

array([0, 1], dtype=int64)

In [41]:
import pickle
import numpy as np

In [42]:
pickle.dump(rf, open("model.sav", 'wb'))

In [61]:

loaded_model = pickle.load(open("model.sav", 'rb'))
predictors = [84,3,1,2,1,1]
#predictors = predictors.reshape(-1,1)
result = loaded_model.predict([[84,3,1,2,1,1]])

print(result)

[0]


In [33]:
X_test

,age,patients_birads,breast_biopsy,comparison_mammogram,family_history,hormone_therapy
7516,84,3,1,2,1,1
13705,67,0,1,2,1,1
21102,60,3,1,2,2,1
10031,67,2,1,2,2,1
18671,68,0,1,2,1,1
...,...,...,...,...,...,...
39643,65,3,1,2,2,1
31431,82,2,1,2,1,1
3036,65,3,1,2,1,1
5604,65,1,2,2,1,1
